In [ ]:
# Testing SkyCoord

In [20]:
from astropy.io import fits
from astropy.table import Table
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import search_around_3d
import numpy as np
import matplotlib.pylab as plt 
import matplotlib.lines as mlines
from matplotlib.legend import Legend
from pythonds.basic.stack import Stack
from math import *

In [21]:
# Reading in data and assigning it to variables even though Greg seems to think it's a waste of time.

# Read in data
hdulist = fits.open('http://portal.nersc.gov/project/cosmo/data/legacysurvey/dr3/external/survey-dr3-specObj-dr13.fits') # survey
hdulist2 = fits.open('https://data.sdss.org/sas/dr13/sdss/spectro/redux/specObj-dr13.fits')
tbdata = hdulist[1].data
tbdata2 = hdulist2[1].data

# Put data in arrays

# Object ID from survey file; value -1 for non-matches
objid = []
objid = tbdata.field('OBJID') 

# Only galaxies included
gal_type = []
gal_type = tbdata.field('TYPE') 

# RA
ra = []
ra = tbdata.field('RA')

# Dec
dec = []
dec = tbdata.field('DEC')

# Number of observations of source from legacy file
# Not cut for LOWZ
obs = []
obsmatch = []
obs = tbdata.field('DECAM_NOBS') 
obsmatch = obs[np.where(((gal_type == 'SIMP') | (gal_type == "DEV") | (gal_type == "EXP")) & (ra >= 241) & (ra <= 246) & (dec >= 6.5) & (dec <= 11.5))] # & (lowz_tar == 1) & (gal_class == 'GALAXY') & (spec == 1 ) & (zwarn_noqso == 0) & (class_noqso == 'GALAXY') & ((survey == 'sdss') | (survey == 'boss')))]

# Put number of observations per filter into arrays that match the filter
uobs = []
gobs = []
robs = []
iobs = []
zobs = []
yobs = []

b = np.array(obsmatch)
uobs = b[:,0]
gobs = b[:,1]
robs = b[:,2]
iobs = b[:,3]
zobs = b[:,4]
yobs = b[:,5]

# Put flux data in an array from legacy file
# Flux has ugrizY, so needs to be divided into 6 arrays
flux =[]
fluxmatch = []

# Flux from DECAM
# Not cut for LOWZ
flux = tbdata.field('DECAM_FLUX')
fluxmatch = flux[np.where(((gal_type == 'SIMP') | (gal_type == "DEV") | (gal_type == "EXP")) & (ra >= 241) & (ra <= 246) & (dec >= 6.5) & (dec <= 11.5))]

# Divide flux arrays into 6 arrays
uflux = []
gflux = []
rflux = []
iflux = []
zflux = []
yflux = []

a = np.array(fluxmatch)
uflux = a[:,0]
gflux = a[:,1]
rflux = a[:,2]
iflux = a[:,3]
zflux = a[:,4]
yflux = a[:,5]

# Class of object
gal_class = []
gal_class = tbdata2.field('CLASS')

# What survey the data is from
survey = []
survey = tbdata2.field('SURVEY')

# SPECPRIMARY; set to 1 for primary observation of object, 0 otherwise
spec = []
spec = tbdata2.field('SPECPRIMARY')

# Bitmask of spectroscopic warning values; need set to 0
zwarn_noqso = []
zwarn_noqso = tbdata2.field('ZWARNING_NOQSO')

# Spectroscopic classification for certain redshift?
class_noqso = []
class_noqso = tbdata2.field('CLASS_NOQSO')

# Redshift of galaxies according to sdss
redshift = []
redshift = tbdata2.field('Z') 

# Array for LOWZ targets
targets = []
targets = tbdata2.field('BOSS_TARGET1')

# Section of code to find LOWZ targets

# Function to find LOWZ targets
def divideBy2(decNumber):
	np.vectorize(decNumber)
	remstack = Stack()
	
	if decNumber == 0: return "0"
	
	while decNumber > 0:
		rem = decNumber % 2
		remstack.push(rem)
		decNumber = decNumber // 2
		
	binString = ""
	while not remstack.isEmpty():
		binString = binString + str(remstack.pop())
			
	return binString

divideBy2Vec = np.vectorize(divideBy2)

a = divideBy2Vec(targets) # gives binary in string form
bin2int = [int(i) for i in a] # converts binary strings to integer
tar = np.array(bin2int) # puts list of integers into numpy array
c = tar % 2 # divide by two again to see if the binary number ends in zero
lowz_tar = np.array(c)

# Number of observations for LOWZ targets
obs_LOWZ = obs[np.where(((gal_type == 'SIMP') | (gal_type == "DEV") | (gal_type == "EXP")) & (ra >= 241) & (ra <= 246) & (dec >= 6.5) & (dec <= 11.5) & (lowz_tar == 1) & (gal_class == 'GALAXY') & (spec == 1 ) & (zwarn_noqso == 0) & (class_noqso == 'GALAXY') & ((survey == 'sdss') | (survey == 'boss')))]

obs_LOWZ_array = np.array(obs_LOWZ)
uobs_LOWZ = obs_LOWZ_array[:,0]
gobs_LOWZ = obs_LOWZ_array[:,1]
robs_LOWZ = obs_LOWZ_array[:,2]
iobs_LOWZ = obs_LOWZ_array[:,3]
zobs_LOWZ = obs_LOWZ_array[:,4]
yobs_LOWZ = obs_LOWZ_array[:,5]

# Flux for LOWZ targets
flux_LOWZ = flux[np.where(((gal_type == 'SIMP') | (gal_type == "DEV") | (gal_type == "EXP")) & (ra >= 241) & (ra <= 246) & (dec >= 6.5) & (dec <= 11.5) & (lowz_tar == 1) & (gal_class == 'GALAXY') & (spec == 1 ) & (zwarn_noqso == 0) & (class_noqso == 'GALAXY') & ((survey == 'sdss') | (survey == 'boss')))]

flux_LOWZ_array = np.array(flux_LOWZ)
uflux_LOWZ = flux_LOWZ_array[:,0]
gflux_LOWZ = flux_LOWZ_array[:,1]
rflux_LOWZ = flux_LOWZ_array[:,2]
iflux_LOWZ = flux_LOWZ_array[:,3]
zflux_LOWZ = flux_LOWZ_array[:,4]
yflux_LOWZ = flux_LOWZ_array[:,5]

In [22]:
# Section of code to find scale factor for distance so I can calculate the area around LRG

# Redshift values
redshift_cut = redshift[np.where((lowz_tar == 1) & (ra >= 241) & (ra <= 246) & (dec >= 6.5) & (dec <= 11.5) & (objid > -1) & (gal_class == 'GALAXY') & (spec == 1 ) & (zwarn_noqso == 0) & (class_noqso == 'GALAXY') & ((survey == 'sdss') | (survey == 'boss')) & ((gal_type == 'DEV') | (gal_type == 'EXP') | (gal_type == 'COMP') | (gal_type == 'SIMP')))]
z = redshift_cut[np.where((gobs_LOWZ >= 3) & (robs_LOWZ >= 3) & (gflux_LOWZ > 0.) & (rflux_LOWZ > 0.))]

# Calculate scale to get areas
H0 = 69.6
WM = 0.286
WV = 0.714
# z = 0.209855

# initialize constants

WR = 0.        # Omega(radiation)
WK = 0.        # Omega curvaturve = 1-Omega(total)
c = 299792.458 # velocity of light in km/sec
Tyr = 977.8    # coefficent for converting 1/H into Gyr
DTT = 0.5      # time from z to now in units of 1/H0
DTT_Gyr = []  # value of DTT in Gyr
age = 0.5      # age of Universe in units of 1/H0
age_Gyr = []  # value of age in Gyr
zage = 0.1     # age of Universe at redshift z in units of 1/H0
zage_Gyr = [] # value of zage in Gyr
DCMR = 0.0     # comoving radial distance in units of c/H0
DCMR_Mpc = [] 
DCMR_Gyr = []
DA = 0.0       # angular size distance
DA_Mpc = []
DA_Gyr = []
kpc_DA = []
DL = 0.0       # luminosity distance
DL_Mpc = []
DL_Gyr = []   # DL in units of billions of light years
V_Gpc = []
a = 1.0        # 1/(1+z), the scale factor of the Universe
az = 0.5       # 1/(1+z(object))

h = H0/100.
WR = 4.165E-5/(h*h)   # includes 3 massless neutrino species, T0 = 2.72528
WK = 1-WM-WR-WV

for j in range(len(z)):
	az = 1.0/(1+1.0*z[j])
	age = 0.
	n=1000         # number of points in integrals
	for i in range(n):
		a = az*(i+0.5)/n
		adot = sqrt(WK+(WM/a)+(WR/(a*a))+(WV*a*a))
		age = age + 1./adot

	zage = az*age/n
	zage_Gyr.append((Tyr/H0)*zage)
	DTT = 0.0
	DCMR = 0.0

	# do integral over a=1/(1+z) from az to 1 in n steps, midpoint rule
	for i in range(n):
		a = az+(1-az)*(i+0.5)/n
		adot = sqrt(WK+(WM/a)+(WR/(a*a))+(WV*a*a))
		DTT = DTT + 1./adot
		DCMR = DCMR + 1./(a*adot)

	DTT = (1.-az)*DTT/n
	DCMR = (1.-az)*DCMR/n
	age = DTT+zage
	age_Gyr.append(age*(Tyr/H0))
	DTT_Gyr.append((Tyr/H0)*DTT)
	DCMR_Gyr.append((Tyr/H0)*DCMR)
	DCMR_Mpc.append((c/H0)*DCMR)

	# tangential comoving distance

	ratio = 1.00
	x = sqrt(abs(WK))*DCMR
	if x > 0.1:
		if WK > 0:
			ratio =  0.5*(exp(x)-exp(-x))/x 
		else:
			ratio = sin(x)/x
	else:
		y = x*x
		if WK < 0: y = -y
		ratio = 1. + y/6. + y*y/120.
	DCMT = ratio*DCMR
	DA = az*DCMT
	DA_Mpc.append((c/H0)*DA)
	kpc_DA.append(DA_Mpc[j]/206.264806)
	DA_Gyr.append((Tyr/H0)*DA)
	DL = DA/(az*az)
	DL_Mpc.append((c/H0)*DL)
	DL_Gyr.append((Tyr/H0)*DL)

	# comoving volume computation

	ratio = 1.00
	x = sqrt(abs(WK))*DCMR
	if x > 0.1:
		if WK > 0:
			ratio = (0.125*(exp(2.*x)-exp(-2.*x))-x/2.)/(x*x*x/3.)
		else:
			ratio = (x/2. - sin(2.*x)/4.)/(x*x*x/3.)
	else:
		y = x*x
		if WK < 0: y = -y
		ratio = 1. + y/5. + (2./105.)*y*y
	VCM = ratio*DCMR*DCMR*DCMR/3.
	V_Gpc.append(4.*pi*((0.001*c/H0)**3)*VCM)
    
print(len(z))

446


In [40]:
# Makes arrays for RA and Dec for LOWZ LRGs only and for all EDR galaxies 

# Creates an array of RA and Dec for the 446 LOWZ LRGs

ra_cut = ra[np.where((lowz_tar == 1) & (ra >= 241) & (ra <= 246) & (dec >= 6.5) & (dec <= 11.5) & (objid > -1) & (gal_class == 'GALAXY') & (spec == 1 ) & (zwarn_noqso == 0) & (class_noqso == 'GALAXY') & ((survey == 'sdss') | (survey == 'boss')) & ((gal_type == 'DEV') | (gal_type == 'EXP') | (gal_type == 'COMP') | (gal_type == 'SIMP')))]
ra_lrg = ra_cut[np.where((gobs_LOWZ >= 3) & (robs_LOWZ >= 3) & (gflux_LOWZ > 0.) & (rflux_LOWZ > 0.))]

dec_cut = dec[np.where((lowz_tar == 1) & (ra >= 241) & (ra <= 246) & (dec >= 6.5) & (dec <= 11.5) & (objid > -1) & (gal_class == 'GALAXY') & (spec == 1 ) & (zwarn_noqso == 0) & (class_noqso == 'GALAXY') & ((survey == 'sdss') | (survey == 'boss')) & ((gal_type == 'DEV') | (gal_type == 'EXP') | (gal_type == 'COMP') | (gal_type == 'SIMP')))]
dec_lrg = dec_cut[np.where((gobs_LOWZ >= 3) & (robs_LOWZ >= 3) & (gflux_LOWZ > 0.) & (rflux_LOWZ > 0.))]

# print(len(ra_lrg))
# print(len(dec_lrg))

# Create an array for RA and Dec for all EDR sources

ra_edr = ra[np.where((ra >= 241) & (ra <= 246) & (dec >= 6.5) & (dec <= 11.5) & ((gal_type == 'DEV') | (gal_type == 'EXP') | (gal_type == 'COMP') | (gal_type == 'SIMP')))]
dec_edr = dec[np.where((ra >= 241) & (ra <= 246) & (dec >= 6.5) & (dec <= 11.5) & ((gal_type == 'DEV') | (gal_type == 'EXP') | (gal_type == 'COMP') | (gal_type == 'SIMP')))]

# print(len(ra_edr))
# print(len(dec_edr))

print(ra_lrg[0])
print(dec_lrg[0])
print(ra_edr[0])
print(dec_edr[0])

# ra_lrg = SkyCoord(ra_lrg, unit="deg", frame="icrs")
# dec_lrg = SkyCoord(dec_lrg, unit="deg", frame="icrs")
# ra_edr = SkyCoord(ra_edr, unit="deg", frame="icrs")
# dec_edr = SkyCoord(dec_edr, unit="deg", frame="icrs")

lrg_zip = list(zip(ra_lrg*u.deg, dec_lrg*u.deg))
edr_zip = list(zip(ra_edr*u.deg, dec_edr*u.deg))

# lrg_zip = np.ndarray(lrg_zip)
# edr_zip = np.ndarray(edr_zip)

print(lrg_zip[0])
print(edr_zip[0])

print(type(edr_zip))

241.073285798
9.38519745779
241.393529136
8.04226458317
(<Quantity 241.07328579839552 deg>, <Quantity 9.385197457791223 deg>)
(<Quantity 241.39352913602988 deg>, <Quantity 8.042264583173143 deg>)
<class 'list'>


In [41]:
# Use SkyCoord to find distance between LRGs and other sources

c1 = []
c2 = []
sep = []

for i in range(len(ra_lrg)):
    c1.append(SkyCoord(ra_lrg[i], dec_lrg[i], unit='deg', frame='icrs'))
    
for i in range(len(ra_edr)):
    c2.append(SkyCoord(ra_edr[i], dec_edr[i], unit='deg', frame='icrs'))
    
    
for i in range(len())
    sep = c1.separation(c2)

print(len(sep))
# print(sep.arcsec)

AttributeError: 'list' object has no attribute 'separation'

In [33]:
dist = search_around_3d(lrg_zip, edr_zip, 50*u.kpc, storekdtree='kdtree_3d')

AttributeError: 'list' object has no attribute 'isscalar'